In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/CS230/Project/git/cs230_project/supervised

Mounted at /content/drive
/content/drive/MyDrive/CS230/Project/git/cs230_project/supervised


In [4]:
import numpy as np
np.random.seed(0)

from keras.models import Model
from keras.layers import Input, Dense, Dropout, LSTM, Activation, Embedding
from keras.initializers import glorot_uniform

from sklearn.model_selection import train_test_split
import tensorflow as tf
# Configure keras backend to run on CPU
# tf.config.set_visible_devices([], 'GPU')

import argparse
from datetime import datetime

from utils import *
from parameters import FLAGS
from preprocessing import review_to_indices, embedding_layer_glove

from models.uni_2_LSTM import Uni_2_LSTM
from models.bi_2_LSTM import Bi_2_LSTM
from models.cnn import CNN1


Parameters:
	dev_sample_percentage: 0.05
	data_file: /content/drive/MyDrive/CS230/Project/git/cs230_project/data/amazon_reviews_us_Electronics_v1_00.tsv
	data_file2: /content/drive/MyDrive/CS230/Project/git/cs230_project/data/amazon_reviews_us_Kitchen_v1_00.tsv
	word2vec_file: /content/drive/MyDrive/CS230/Project/git/cs230_project/data/glove.6B.100d.txt
	min_total_votes: 5
	max_review_word_count: 200
	keep_start_of_longer_reviews: True
	batch_size: 32
	num_epochs: 50
	debug_mode: False


In [5]:
X, Y = load_data(FLAGS.data_file,
                 FLAGS.min_total_votes,
                 FLAGS.max_review_word_count,
                 FLAGS.keep_start_of_longer_reviews)

X_train, X_dev, Y_train, Y_dev = train_test_split(X, Y, test_size=FLAGS.dev_sample_percentage, random_state=42)

Loading data ...
Chunk loaded. Found 291203 data points with >= 5 total votes.
Discarded 2799900 data points.


In [ ]:
print(f'Examples in X_train: {X_train.shape[0]}')
print(f'Examples in X_dev:   {X_dev.shape[0]}')

In [ ]:
for idx in range(3):
    print(X_train[idx], '\n', Y_train[idx])
    print('\n\n')

print('#'*50, '\n\n')

for idx in range(3):
    print(X_dev[idx], '\n', Y_dev[idx])
    print('\n\n')

In [6]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs(FLAGS.word2vec_file)

Creating word embeddings matrix ...
Done


In [ ]:
X1 = np.array(["funny lol", "lets play baseball", "food is ready for you"])
X1_indices = review_to_indices(X1, word_to_index, max_review_word_count=5)
print("X1 =", X1)
print("X1_indices =\n", X1_indices)

In [5]:
model_biLSTM = Bi_2_LSTM((FLAGS.max_review_word_count, ),
                         word_to_vec_map,
                         word_to_index,
                         lstm_units=[32, 64])
model_biLSTM.summary()

Model: "bi_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200)]             0         
                                                                 
 embedding (Embedding)       (None, 200, 100)          40000100  
                                                                 
 bidirectional (Bidirectiona  (None, 200, 64)          34048     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 200, 64)           0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              66048     
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 128)               0 

In [ ]:
model_cnn1 = CNN1((FLAGS.max_review_word_count, ),
                  word_to_vec_map,
                  word_to_index,
                  FLAGS)
model_cnn1.summary()

In [ ]:
import matplotlib.pyplot as plt
plt.hist([len(X_train[i].split(' ')) for i in range(X_train.shape[0])], bins=40)
plt.xlim([0, 200])
plt.show()

plt.hist(Y_train, bins=100)
plt.xlim([0, 1])
plt.show()

In [ ]:
np.var(Y_train)

In [6]:
# model_biLSTM.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])
# model_cnn1.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])

model_biLSTM.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                     loss=tf.keras.losses.MeanAbsoluteError(),
                     metrics=['mse', 'mean_absolute_error'])

In [7]:
%pwd

'/content/drive/MyDrive/CS230/Project/git/cs230_project/supervised'

In [7]:
X_train_indices = review_to_indices(X_train,
                                    word_to_index,
                                    FLAGS.max_review_word_count)
X_dev_indices   = review_to_indices(X_dev,
                                    word_to_index,
                                    FLAGS.max_review_word_count)

# checkpoint_filepath = '/content/drive/MyDrive/CS230/Project/git/cs230_project/supervised/trained_models/bi_60epochs.ckpt'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    save_freq='epoch')

In [ ]:
model_biLSTM.fit(X_train_indices, Y_train,
                 epochs=FLAGS.num_epochs,
                 batch_size=64,
                 shuffle=True,
                 validation_data=(X_dev_indices, Y_dev),
                 callbacks=[model_checkpoint_callback])
# Epoch 60/60
# 2021/2021 [==============================] - 57s 28ms/step - loss: 0.1100 - mse: 0.0307 - mean_absolute_error: 0.1100 - val_loss: 0.1740 - val_mse: 0.0745 - val_mean_absolute_error: 0.1740

Epoch 1/50
4323/4323 [==============================] - ETA: 0s - loss: 0.1856 - mse: 0.0646 - mean_absolute_error: 0.1832

4323/4323 [==============================] - 151s 35ms/step - loss: 0.1856 - mse: 0.0646 - mean_absolute_error: 0.1832 - val_loss: 0.1830 - val_mse: 0.0619 - val_mean_absolute_error: 0.1802
Epoch 2/50
4323/4323 [==============================] - 123s 28ms/step - loss: 0.1835 - mse: 0.0635 - mean_absolute_error: 0.1812 - val_loss: 0.1836 - val_mse: 0.0664 - val_mean_absolute_error: 0.1811
Epoch 3/50
4322/4323 [============================>.] - ETA: 0s - loss: 0.1819 - mse: 0.0626 - mean_absolute_error: 0.1797

4323/4323 [==============================] - 147s 34ms/step - loss: 0.1819 - mse: 0.0626 - mean_absolute_error: 0.1797 - val_loss: 0.1797 - val_mse: 0.0606 - val_mean_absolute_error: 0.1780
Epoch 4/50
4323/4323 [==============================] - ETA: 0s - loss: 0.1806 - mse: 0.0619 - mean_absolute_error: 0.1784

4323/4323 [==============================] - 145s 33ms/step - loss: 0.1806 - mse: 0.0619 - mean_absolute_error: 0.1784 - val_loss: 0.1795 - val_mse: 0.0606 - val_mean_absolute_error: 0.1772
Epoch 5/50
4323/4323 [==============================] - ETA: 0s - loss: 0.1796 - mse: 0.0612 - mean_absolute_error: 0.1774

4323/4323 [==============================] - 148s 34ms/step - loss: 0.1796 - mse: 0.0612 - mean_absolute_error: 0.1774 - val_loss: 0.1789 - val_mse: 0.0588 - val_mean_absolute_error: 0.1768
Epoch 6/50
4323/4323 [==============================] - 122s 28ms/step - loss: 0.1787 - mse: 0.0607 - mean_absolute_error: 0.1765 - val_loss: 0.1802 - val_mse: 0.0616 - val_mean_absolute_error: 0.1778
Epoch 7/50
4323/4323 [==============================] - 123s 28ms/step - loss: 0.1778 - mse: 0.0603 - mean_absolute_error: 0.1756 - val_loss: 0.1818 - val_mse: 0.0660 - val_mean_absolute_error: 0.1793
Epoch 8/50
4323/4323 [==============================] - 121s 28ms/step - loss: 0.1772 - mse: 0.0599 - mean_absolute_error: 0.1751 - val_loss: 0.1789 - val_mse: 0.0613 - val_mean_absolute_error: 0.1767
Epoch 9/50
4322/4323 [============================>.] - ETA: 0s - loss: 0.1764 - mse: 0.0595 - mean_absolute_error: 0.1742

4323/4323 [==============================] - 148s 34ms/step - loss: 0.1764 - mse: 0.0595 - mean_absolute_error: 0.1742 - val_loss: 0.1776 - val_mse: 0.0610 - val_mean_absolute_error: 0.1755
Epoch 10/50
4323/4323 [==============================] - 125s 29ms/step - loss: 0.1759 - mse: 0.0592 - mean_absolute_error: 0.1738 - val_loss: 0.1815 - val_mse: 0.0580 - val_mean_absolute_error: 0.1793
Epoch 11/50
4323/4323 [==============================] - 120s 28ms/step - loss: 0.1754 - mse: 0.0589 - mean_absolute_error: 0.1732 - val_loss: 0.1804 - val_mse: 0.0619 - val_mean_absolute_error: 0.1780
Epoch 12/50
4323/4323 [==============================] - 122s 28ms/step - loss: 0.1748 - mse: 0.0586 - mean_absolute_error: 0.1726 - val_loss: 0.1782 - val_mse: 0.0607 - val_mean_absolute_error: 0.1758
Epoch 13/50
4323/4323 [==============================] - ETA: 0s - loss: 0.1744 - mse: 0.0585 - mean_absolute_error: 0.1722

4323/4323 [==============================] - 148s 34ms/step - loss: 0.1744 - mse: 0.0585 - mean_absolute_error: 0.1722 - val_loss: 0.1763 - val_mse: 0.0599 - val_mean_absolute_error: 0.1741
Epoch 14/50
4323/4323 [==============================] - 122s 28ms/step - loss: 0.1740 - mse: 0.0581 - mean_absolute_error: 0.1717 - val_loss: 0.1769 - val_mse: 0.0588 - val_mean_absolute_error: 0.1747
Epoch 15/50
4323/4323 [==============================] - 122s 28ms/step - loss: 0.1735 - mse: 0.0579 - mean_absolute_error: 0.1713 - val_loss: 0.1778 - val_mse: 0.0605 - val_mean_absolute_error: 0.1753
Epoch 16/50
4322/4323 [============================>.] - ETA: 0s - loss: 0.1732 - mse: 0.0578 - mean_absolute_error: 0.1710

4323/4323 [==============================] - 148s 34ms/step - loss: 0.1732 - mse: 0.0578 - mean_absolute_error: 0.1710 - val_loss: 0.1762 - val_mse: 0.0607 - val_mean_absolute_error: 0.1741
Epoch 17/50
4323/4323 [==============================] - 131s 30ms/step - loss: 0.1729 - mse: 0.0575 - mean_absolute_error: 0.1707 - val_loss: 0.1781 - val_mse: 0.0602 - val_mean_absolute_error: 0.1758
Epoch 18/50
4323/4323 [==============================] - 128s 29ms/step - loss: 0.1725 - mse: 0.0574 - mean_absolute_error: 0.1702 - val_loss: 0.1770 - val_mse: 0.0607 - val_mean_absolute_error: 0.1747
Epoch 19/50
4323/4323 [==============================] - 128s 30ms/step - loss: 0.1722 - mse: 0.0571 - mean_absolute_error: 0.1700 - val_loss: 0.1769 - val_mse: 0.0584 - val_mean_absolute_error: 0.1749
Epoch 20/50
4323/4323 [==============================] - 128s 30ms/step - loss: 0.1719 - mse: 0.0570 - mean_absolute_error: 0.1696 - val_loss: 0.1772 - val_mse: 0.0608 - val_mean_absolute_error: 0.1750
Ep

4323/4323 [==============================] - 153s 35ms/step - loss: 0.1717 - mse: 0.0570 - mean_absolute_error: 0.1694 - val_loss: 0.1760 - val_mse: 0.0591 - val_mean_absolute_error: 0.1742
Epoch 22/50
4323/4323 [==============================] - 133s 31ms/step - loss: 0.1712 - mse: 0.0567 - mean_absolute_error: 0.1689 - val_loss: 0.1796 - val_mse: 0.0626 - val_mean_absolute_error: 0.1773
Epoch 23/50
4323/4323 [==============================] - 131s 30ms/step - loss: 0.1710 - mse: 0.0565 - mean_absolute_error: 0.1687 - val_loss: 0.1776 - val_mse: 0.0604 - val_mean_absolute_error: 0.1751
Epoch 24/50
4323/4323 [==============================] - 128s 30ms/step - loss: 0.1708 - mse: 0.0565 - mean_absolute_error: 0.1685 - val_loss: 0.1787 - val_mse: 0.0602 - val_mean_absolute_error: 0.1765
Epoch 25/50
4323/4323 [==============================] - 129s 30ms/step - loss: 0.1706 - mse: 0.0564 - mean_absolute_error: 0.1682 - val_loss: 0.1763 - val_mse: 0.0602 - val_mean_absolute_error: 0.1743
Ep

In [8]:
model_biLSTM_loadded = tf.keras.models.load_model(checkpoint_filepath)
model_biLSTM_loadded.evaluate(X_dev_indices, Y_dev, verbose=2)

456/456 - 44s - loss: 0.1760 - mse: 0.0591 - mean_absolute_error: 0.1742 - 44s/epoch - 96ms/step


[0.17598721385002136, 0.059060122817754745, 0.17418918013572693]